<a href="https://colab.research.google.com/github/naufalperdana/ANN_Prediksi_Tutup_Rekening_Bank/blob/main/ann_sederhana_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pendeteksian Tutup Rekening Nasabah Bank  (Churn)

## Bagian 1 - Data Preprocessing

### Importing the libraries

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd

Mengetahui Versi of Tensorflow

In [2]:
tf.__version__

'2.4.0'

### Importing dataset


In [4]:
dataset = pd.read_csv('DataNasabah.csv')


**Melihat Dataset**

In [5]:
dataset.head()

,Nomer,IDKostumer,Nama,NilaiKredit,Provinsi,JenisKelamin,Umur,tenor,saldo,jumlahProduk,PunyaKartuKredit,NasabahAktif,PerkiraanPendapatan,TutupRekening
0,1,15634602,ERWIN TJAHJONO,619,DKI Jakarta,Laki-Laki,42,2,0,1,1,1,10134888000,1
1,2,15647311,DAVIANDRIE ANDIKA BAHROENY,608,Jawa Barat,Laki-Laki,41,1,8380786000,1,0,1,11254258000,0
2,3,15619304,ELAN KURNIAWAN,502,DKI Jakarta,Laki-Laki,42,8,1596608000,3,1,0,11393157000,1
3,4,15701354,AYU DWI CAHYANING MUKTI,699,DKI Jakarta,Perempuan,39,1,0,2,0,0,9382663000,0
4,5,15737888,WAHYOEDIN,850,Jawa Barat,Laki-Laki,43,2,12551082000,1,1,1,790841000,0


In [6]:
dataset.describe()

,Nomer,IDKostumer,NilaiKredit,Umur,tenor,saldo,jumlahProduk,PunyaKartuKredit,NasabahAktif,PerkiraanPendapatan,TutupRekening
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000,10000.00000,10000.000000,1.000000e+04,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,6.980471e+09,1.530200,0.70550,0.515100,9.089385e+09,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,6.285896e+09,0.581654,0.45584,0.499797,6.128208e+09,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000e+00,1.000000,0.00000,0.000000,5.230000e+05,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000e+00,1.000000,0.00000,0.000000,3.336751e+09,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,8.668510e+09,1.000000,1.00000,1.000000,8.890572e+09,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,1.246215e+10,2.000000,1.00000,1.000000,1.439529e+10,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,2.508981e+10,4.000000,1.00000,1.000000,1.999925e+10,1.000000


In [7]:
# Memeriksa tipe data dan nilai null
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Nomer                10000 non-null  int64 
 1   IDKostumer           10000 non-null  int64 
 2   Nama                 10000 non-null  object
 3   NilaiKredit          10000 non-null  int64 
 4   Provinsi             10000 non-null  object
 5   JenisKelamin         10000 non-null  object
 6   Umur                 10000 non-null  int64 
 7   tenor                10000 non-null  int64 
 8   saldo                10000 non-null  int64 
 9   jumlahProduk         10000 non-null  int64 
 10  PunyaKartuKredit     10000 non-null  int64 
 11  NasabahAktif         10000 non-null  int64 
 12  PerkiraanPendapatan  10000 non-null  int64 
 13  TutupRekening        10000 non-null  int64 
dtypes: int64(11), object(3)
memory usage: 1.1+ MB


In [8]:
# Mengabaikan 3 kolom pertama dan kolom terakhir, karena bukan fitur yang berguna.
# Indek dimulai dari 0.
x = dataset.iloc[:, 3:-1].values

# ambil kolom terakhir untuk mendapatkan sebagai nilai penentu(kelas tutup rekening / ada rekening).
# Just get the last column of dataset for the dependent variable.
y = dataset.iloc[:, -1].values

In [9]:
print(x)

[[619 'DKI Jakarta' 'Laki-Laki' ... 1 1 10134888000]
 [608 'Jawa Barat' 'Laki-Laki' ... 0 1 11254258000]
 [502 'DKI Jakarta' 'Laki-Laki' ... 1 0 11393157000]
 ...
 [709 'DKI Jakarta' 'Perempuan' ... 0 1 4208558000]
 [772 'Jawa Tengah' 'Laki-Laki' ... 1 0 9288852000]
 [792 'DKI Jakarta' 'Perempuan' ... 1 0 3819078000]]


In [10]:
print(y)

[1 0 1 ... 1 1 0]


### Mengkodekan kategori data

Melabeli kode kolom "JenisKelamin" 

In [11]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
# Ambil semua baris untuk kolom ke-2 dan dikodekan.
x[:, 2] = le.fit_transform(x[:, 2])

In [12]:
print(x)

[[619 'DKI Jakarta' 0 ... 1 1 10134888000]
 [608 'Jawa Barat' 0 ... 0 1 11254258000]
 [502 'DKI Jakarta' 0 ... 1 0 11393157000]
 ...
 [709 'DKI Jakarta' 1 ... 0 1 4208558000]
 [772 'Jawa Tengah' 0 ... 1 0 9288852000]
 [792 'DKI Jakarta' 1 ... 1 0 3819078000]]


One Hot Encoding Kolom "Provinsi" 

In [13]:
# dibuatkan one hot coding untuk tempat provinsi karena hal tersebut tidak berhubungan dengan mereka sendiri seperti halnya jenis kelamin.
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
x = np.array(ct.fit_transform(x))

In [14]:
print (x)

[[1.0 0.0 0.0 ... 1 1 10134888000]
 [0.0 1.0 0.0 ... 0 1 11254258000]
 [1.0 0.0 0.0 ... 1 0 11393157000]
 ...
 [1.0 0.0 0.0 ... 0 1 4208558000]
 [0.0 0.0 1.0 ... 1 0 9288852000]
 [1.0 0.0 0.0 ... 1 0 3819078000]]


### Membagi dataset menjadi data ajar dan data tes

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 1)

### Skalar Fitur

In [16]:
# membuat skalar fitur untuk semua fitur yang ada dalam data ajar dan tes.
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
# Penyesuaian Skalar hanya pada data ajar untuk mencegah kebocoran informasi.
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Bagian 2 - Membangun ANN

### Penyiapkan ANN

In [17]:
# Membuat variabel yang mempresentasikan  ANN sebagai contoh dari kelas sekuensial yang menyiapkan ANN.
ann = tf.keras.models.Sequential()

### Menambahkan layer masukan dan layer pertama tersembunyi

In [18]:
# prosedur add() merupakan bagian kelas sekuensial.
# penambahan keseluruhan layer yang terhubung (tersembunyi), yang menjari objek baru untuk kelas yang padat.
# argumen dari dense() - 
# 'units' - merupakan penyebutan hyperparameters dan merupakan syaraf tersembunyi.
# 'activation' - fungsi aktifasi dalam layer yang tersembunyi harus sebagai fungsi aktifasi penyearah.
ann.add(tf.keras.layers.Dense(units=6, activation='relu')) # 'relu' - nama kode untuk fungsi aktifasi penyearah.

### Menambahkan layar tersembunyi kedua

In [19]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu')) 

### Menambahkan layer keluaran

In [20]:
# Sebagaimana Layer Keluaran merupakan suatu binari yang hanya 1 syaraf untuk dicocokan dengan layer keluaran, sehingga 'units' = 1
# dibutuhkan fungsi aktifasi sigmoid, karena hal ini memberikan prediksi sebagaimana peluang probalilitasnya.# jika akan memprediksi variabel kategori, maka activation = 'sigmax'
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid')) 

## Bagian 3 - Pelatihan ANN

### Mengkompilasi ANN

In [21]:
# Argumen - 
# 'optimizer' - dipilih optimasi 'adam' karena dapat memberikan performa penurunan gradien stokastik. 
#  Penurunan gradien stokastik akan mengoptimalkan bobot untuk meminimalkan kerugian antara nilai aktual dan prediksi.
# 'loss' - Saat dilakukan klasifikasi biner dalam memprediksi variabel biner gunakan 'binary_crossentropy' dan 
#  jika memiliki variabel kategorikal untuk memprediksi digunakan 'categorical_crossentropy'
# 'metrics' - 
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Pelatihan ANN pada data ajar

In [22]:
# Pembelajaran batch, ukuran batch default untuk dibandingkan dengan hasil aktual = 32
ann.fit(X_train, y_train, batch_size=32, epochs=100)
# Seperti yang bisa dilihat, akurasi berkumpul di sekitar 0,86 di sekitar epoch = 26, jadi tidak dibutuhkan sampai 100 epoch.

Epoch 1/100
250/250 [==============================] - 2s 2ms/step - loss: 0.6661 - accuracy: 0.5865
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4912 - accuracy: 0.7956
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4551 - accuracy: 0.8037
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4352 - accuracy: 0.8126
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4276 - accuracy: 0.8186
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4160 - accuracy: 0.8177
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4073 - accuracy: 0.8231
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4029 - accuracy: 0.8264
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3959 - accuracy: 0.8307
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3899 - accura

## Bagian 4 - Membuat Prediksi dan mengevaluasi model

### Memprediksi Hasil dari observasi tunggal

Menggunakan model ANN ini untuk memprediksi apakah nasabah dengan informasi berikut akan menutup rekening : 

Provinsi: DKI Jakarta

Nilai Kredit: 600

Jenis Kelamin: Laki-Laki

Umur: 40 years old

Tenor: 3 years

Saldo: \$ 60000

Jumlah Produk: 2

Apakah punya kartu kredit? Yes

nasabah aktif: Yes

perkiraan pendapatan: \$ 50000

Jadi, haruskah diucapkan selamat tinggal kepada pelanggan tersebut?

**Solution**

In [23]:
# Sebagaimana digunakan fungsi sigmoid, diperoleh probabilitas nasabah ini tutup rekening :.
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))

[[0.02820033]]


Namum, model ANN ini memprediksi nasabah tersebut masih memiliki rekening di bank!

**Catatan Penting 1:** Perhatikan bahwa nilai fitur yang diinputkan dalam kurung siku pasangan ganda. karena metoda "predict" selalu mengharapkan array 2D array sebagai input. Dan menaruh nilai tersebut dalam kurung siku pasangan ganda membuat input menjadi array 2D.

**Catatan Penting 2:** Perhatikan juga bahwa Provinsi "DKI Jakarta" tidak sebagai input string pada kolom tetapi sebagai "1, 0, 0" dalam 3 kolom pertama. Hal itu dikarenakan metoda prediksi mengharapkan nilai one-hot-encoded sebagai inputan, dan seperti yang terlihat pada baris pertama dari fitur matrik X, "DKI Jakarta" dikodekan sebagai "1, 0, 0". Dan selalu perhatikan untuk memasukan nilai ini dalam 3 kolom pertama, karena variabel buatan selalu dibuat dalam kolom pertama.

### Memprediksi data tes

In [24]:
# Ini merupakan probabilitas dari prediksi
y_predicted = ann.predict(X_test)
# Konversi probabilitas prediksi menjadi hasil binari
y_predicted = (y_predicted > 0.5)
print(np.concatenate((y_predicted.reshape(len(y_predicted),1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 0]
 [0 0]
 ...
 [0 0]
 [0 0]
 [1 0]]


### Membuat Confusion Matrix

In [25]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_predicted)
print(cm)
accuracy_score(y_test, y_predicted)

[[1526   59]
 [ 219  196]]


0.861

Sehingga, Akurasi = 86,1%

Total Prediksi benar bahwa nasabah masih punya rekening = 1526

Total Prediksi benar bahwa nasabah tutup rekening = 59

Total Prediksi salah bahwa nasabah masih punya rekening = 219

Total Prediksi salah bahwa nasabah tutup rekening = 196